# 8-9. 프로젝트 - Movielens 영화 추천 실습


추천의 MNIST라고 부를만한 Movielens 데이터


* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 데이터 준비와 전처리

데이터를 다운로드받고 준비한 경로로 옮겨주고 압출을 풉니다.

``` terminal
wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
mv ml-1m.zip ~/aiffel/recommendata_iu/data
cd ~/aiffel/recommendata_iu/data
unzip ml-1m.zip
```

In [2]:
#데이터 불로오기 
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
# 변경된 내용 확인 
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

timestamp는 우리 추천시스템에서 그다지 의미가 없으니 제외합니다.

In [5]:
ratings = ratings[['user_id','movie_id','count']]
ratings.tail()

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 로딩 
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()
movies.tail()

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


데이터를 불러왔으니 한번 살펴보도록 하겠습니다.

## 데이터 분석
ratings에 있는 유니크한 영화 개수         
rating에 있는 유니크한 사용자 수                  
가장 인기 있는 영화 30개(인기순)               

In [7]:
for c in ratings.columns:
    print('{} : {}'.format(c, len(ratings.loc[pd.isnull(ratings[c]), c].values)))

user_id : 0
movie_id : 0
count : 0


In [8]:
for c in movies.columns:
    print('{} : {}'.format(c, len(movies.loc[pd.isnull(movies[c]), c].values)))

movie_id : 0
title : 0
genre : 0


일단 missing value는 없는것으로 확인하였습니다.

In [9]:
ratings['user_id'].nunique(),ratings['movie_id'].nunique(),movies['movie_id'].nunique()

(6039, 3628, 3883)

유니크한 user_id는 6039 만큼 있으며 유니크한 영화는 3628개 만큼있습니다.      
moveis에는 더 많은 3883 만큼의 movie_id가 있는데 ratings에는 평점 3점미만의  
영화를 삭제했기 때문에 차이가 있는것같습니다.

In [10]:
a = ratings.groupby(by = "movie_id")["count"].sum().sort_values().tail(30)
a

movie_id
1214     8244
2716     8282
1        8475
1240     8571
356      8679
1097     8696
1265     8727
2997     8964
296      9053
1580     9057
2396     9503
1617     9515
480      9601
1197     9866
858      9965
1270    10081
318     10085
110     10125
527     10317
608     10465
589     10513
2762    10703
2571    10903
593     11096
1198    11179
1210    11303
2028    11348
1196    12648
260     13178
2858    14449
Name: count, dtype: int64

가장 movie_id별로 가장 count의 합이 큰 영화 목록입니다.

In [11]:
most_pop_id_count = a.to_dict()
most_pop_id_count

{1214: 8244,
 2716: 8282,
 1: 8475,
 1240: 8571,
 356: 8679,
 1097: 8696,
 1265: 8727,
 2997: 8964,
 296: 9053,
 1580: 9057,
 2396: 9503,
 1617: 9515,
 480: 9601,
 1197: 9866,
 858: 9965,
 1270: 10081,
 318: 10085,
 110: 10125,
 527: 10317,
 608: 10465,
 589: 10513,
 2762: 10703,
 2571: 10903,
 593: 11096,
 1198: 11179,
 1210: 11303,
 2028: 11348,
 1196: 12648,
 260: 13178,
 2858: 14449}

In [12]:
most_pop_movie_id = []
for k in most_pop_id_count.keys():
    most_pop_movie_id.append(k)

print(most_pop_movie_id)

[1214, 2716, 1, 1240, 356, 1097, 1265, 2997, 296, 1580, 2396, 1617, 480, 1197, 858, 1270, 318, 110, 527, 608, 589, 2762, 2571, 593, 1198, 1210, 2028, 1196, 260, 2858]


dict 꼴로 만들었다가 인기작 movie_id만 뽑아내고 movies에서 그 move_id만       
뽑아냈습니다.

In [13]:
merged_movie = pd.merge(ratings,movies,how= 'left', on = 'movie_id')
merged_movie

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
836473,6040,1090,3,Platoon (1986),Drama|War
836474,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,Sophie's Choice (1982),Drama


하다가 그냥 통합해서 해도 될꺼같아서 merge를 해보았습니다.       
movies에는 ratings에는 없는 영화가 있어서 left 방식으로 merge 했습니다.

In [14]:
merged_movie.groupby(by = ["movie_id","title"])["count"].sum().sort_values().tail(30)

movie_id  title                                                
1214      Alien (1979)                                              8244
2716      Ghostbusters (1984)                                       8282
1         Toy Story (1995)                                          8475
1240      Terminator, The (1984)                                    8571
356       Forrest Gump (1994)                                       8679
1097      E.T. the Extra-Terrestrial (1982)                         8696
1265      Groundhog Day (1993)                                      8727
2997      Being John Malkovich (1999)                               8964
296       Pulp Fiction (1994)                                       9053
1580      Men in Black (1997)                                       9057
2396      Shakespeare in Love (1998)                                9503
1617      L.A. Confidential (1997)                                  9515
480       Jurassic Park (1993)                              

American Beauty가 가장 인기있는 영화였습니다. 그다음은 스타워즈 시리즈가 있었습니다.

In [15]:
movies[movies['movie_id'].isin(most_pop_movie_id)]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
293,296,Pulp Fiction (1994),Crime|Drama
315,318,"Shawshank Redemption, The (1994)",Drama
352,356,Forrest Gump (1994),Comedy|Romance|War
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
523,527,Schindler's List (1993),Drama|War
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller


그결과로 movies에서도 뽑아서 인기영화를 30개만 뽑아보았습니다.(주의: 인기순아님)

## 제가 선호하는 영화 추가



In [16]:
# 주라기공원, 브레이브하트,토이스토리,쇼생크 탈출,  라이언 일병 구하기를 골랐습니다. 
my_favorite = ['Jurassic Park (1993)' , 'Braveheart (1995)' ,'Toy Story (1995)' ,'Shawshank Redemption, The (1994)' ,'Saving Private Ryan (1998)']

a = movies[movies['title'].isin(my_favorite)]['movie_id'].values
print(a)

[   1  110  318  480 2028]


In [17]:
print(type(ratings))

ratings.head()

<class 'pandas.core.frame.DataFrame'>


,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


저는 hayaseleu라는 user_id를 가지고 제가 위에서 고른 영화를 5count씩봤다고 합시다. 

In [18]:
# 'hayaseleu'이라는 user_id이고 위 영화를 5점씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['hayaseleu']*5, 'movie_id': a, 'count':[5]*5})

if not ratings.isin({'user_id':['hayaseleu']})['user_id'].any():  # user_id에 'hayaseleu'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_playlist 데이터를 추가해 줍니다. 

ratings.head(10)

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [19]:
ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,hayaseleu,1,5
1,hayaseleu,110,5
2,hayaseleu,318,5
3,hayaseleu,480,5
4,hayaseleu,2028,5


In [20]:
merged = pd.merge(ratings,movies,how= 'left', on = 'movie_id')
merged

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
836478,hayaseleu,1,5,Toy Story (1995),Animation|Children's|Comedy
836479,hayaseleu,110,5,Braveheart (1995),Action|Drama|War
836480,hayaseleu,318,5,"Shawshank Redemption, The (1994)",Drama
836481,hayaseleu,480,5,Jurassic Park (1993),Action|Adventure|Sci-Fi


아까 836478행에비해 5개 만큼 상승한걸 확인할 수있었습니다.

In [21]:
data = merged[['user_id','movie_id','count']]

## CSR matrix를 직접 만들보기


In [26]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

In [27]:
user_unique


array([1, 2, 3, ..., 6039, 6040, 'hayaseleu'], dtype=object)

In [28]:
movie_unique

array([1193,  661,  914, ...,  690, 2909, 1360])

In [35]:
# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_id_to_idx = {v:k for k,v in enumerate(moive_unique)}

print(user_to_idx)
# 인덱싱이 잘 되었는지 확인해 봅니다.  
print(user_to_idx['hayaseleu'])
# 6040명의 유저 중 0부터 시작해서 마지막에 부여된 index니 6039이 나와야합니다.


{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 120, 122: 12

In [36]:
print(movie_id_to_idx)

{1193: 0, 661: 1, 914: 2, 3408: 3, 2355: 4, 1197: 5, 1287: 6, 2804: 7, 594: 8, 919: 9, 595: 10, 938: 11, 2398: 12, 2918: 13, 1035: 14, 2791: 15, 2687: 16, 2018: 17, 3105: 18, 2797: 19, 2321: 20, 720: 21, 1270: 22, 527: 23, 2340: 24, 48: 25, 1097: 26, 1721: 27, 1545: 28, 745: 29, 2294: 30, 3186: 31, 1566: 32, 588: 33, 1907: 34, 783: 35, 1836: 36, 1022: 37, 2762: 38, 150: 39, 1: 40, 1961: 41, 1962: 42, 2692: 43, 260: 44, 1028: 45, 1029: 46, 1207: 47, 2028: 48, 531: 49, 3114: 50, 608: 51, 1246: 52, 1357: 53, 3068: 54, 1537: 55, 647: 56, 2194: 57, 648: 58, 2268: 59, 2628: 60, 1103: 61, 2916: 62, 3468: 63, 1210: 64, 1792: 65, 1687: 66, 3578: 67, 2881: 68, 3030: 69, 1217: 70, 2126: 71, 3108: 72, 3035: 73, 1253: 74, 1610: 75, 292: 76, 2236: 77, 3071: 78, 368: 79, 1259: 80, 3147: 81, 1544: 82, 1293: 83, 1188: 84, 3255: 85, 3257: 86, 110: 87, 2278: 88, 2490: 89, 1834: 90, 3471: 91, 589: 92, 1690: 93, 3654: 94, 2852: 95, 1945: 96, 982: 97, 1873: 98, 2858: 99, 1225: 100, 515: 101, 442: 102, 2312:

In [38]:
print(movie_id_to_idx[1])
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

40


In [39]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data): 
    # 모든 row가 정상적으로 인덱싱되었다면
    print('indexing OK')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

indexing OK


/home/ssac4/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [42]:
# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_id_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('artist column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

data

artist column indexing OK!!


/home/ssac4/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
836478,6039,40,5
836479,6039,87,5
836480,6039,157,5
836481,6039,107,5


In [43]:
from scipy.sparse import csr_matrix
num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()
data, num_user, num_movie

(        user_id  movie_id  count
 0             0         0      5
 1             0         1      3
 2             0         2      3
 3             0         3      4
 4             0         4      5
 ...         ...       ...    ...
 836478     6039        40      5
 836479     6039        87      5
 836480     6039       157      5
 836481     6039       107      5
 836482     6039        48      5
 
 [836483 rows x 3 columns],
 6040,
 3628)

In [44]:
data.movie_id

0           0
1           1
2           2
3           3
4           4
         ... 
836478     40
836479     87
836480    157
836481    107
836482     48
Name: movie_id, Length: 836483, dtype: int64

In [45]:
csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)))


csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

이제 MF model을 구성하여 내가 좋아할만한 영화를 추천합니다. 

## 모델학습 

In [46]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [47]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, use_gpu=False, iterations=200, dtype=np.float32)
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.) # factor 500이나 ,1000 놓으면 오버피팅인지확인좀
csr_data_transpose = csr_data.T
csr_data_transpose
# 모델 훈련
als_model.fit(csr_data_transpose)


  0%|          | 0/200 [00:00<?, ?it/s]

In [50]:
a #제가 좋아하는 영화의 movie id입니다.

movies[movies['movie_id'].isin(a)]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
108,110,Braveheart (1995),Action|Drama|War
315,318,"Shawshank Redemption, The (1994)",Drama
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1959,2028,Saving Private Ryan (1998),Action|Drama|War


그중 Toy Story (1995)와 저의 선호도를 살펴보겠습니다.

In [48]:
hayaseleu, toy_story = user_to_idx['hayaseleu'], movie_id_to_idx[1]
hayaseleu_vector, toy_story_vector = als_model.user_factors[hayaseleu], als_model.item_factors[toy_story]

print('슝=3')
hayaseleu_vector 
toy_story_vector
# 제 코드와 Toy Story (1995)를 내적하는 코드
np.dot(hayaseleu_vector , toy_story_vector)


슝=3


0.91276944

저와 toy_stroy_vector의 내적은 0.91로 계산되었습니다. 1이 최고점이라고할때 괜찮게 적합된것같습니다.  
한번 Godfather, The (1972) 와 비교해보록 하겠습니다. 

In [79]:
movies[movies['title'] == ('Godfather, The (1972)')].movie_id.values.tolist()

[858]

Godfather, The (1972)의 movie_id는 858입니다. 

In [59]:
godfather = movie_id_to_idx[858]
godfather_vector = als_model.item_factors[godfather]
np.dot(hayaseleu_vector, godfather_vector)

-0.009455916

-0.009455916의 값이 나왔습니다. 저는 godfather와 잘 맞지않는다고 예측된것입니다. 

## 좋아하는 영화와 비슷한 영화 찾기 

제가 좋아하는 Braveheart (1995)와 비슷한 영화를 찾아보겠습니다. 브레이브하트는 스코틀랜드 독립 전쟁을 다룬 역사영화입니다.
Braveheart (1995)의 movie id는 110입니다. 

In [61]:
favorite_movie = 110
movie_id = movie_id_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie


[(87, 1.0),
 (3549, 0.20940584),
 (3499, 0.18365549),
 (3150, 0.15696874),
 (3582, 0.15458229),
 (2293, 0.14718468),
 (3446, 0.14573805),
 (3390, 0.14570312),
 (3554, 0.14501129),
 (2086, 0.14252515),
 (3481, 0.14203466),
 (3263, 0.14147931),
 (48, 0.14068228),
 (2695, 0.14038229),
 (3181, 0.13985391)]

In [85]:
#movie_id_to_idx 를 뒤집어, index로부터 movie_id를 얻는 dict를 생성합니다. 
idx_to_movie_id = {v:k for k,v in movie_id_to_idx.items()}
similar_braveheart = [idx_to_movie_id[i[0]] for i in similar_movie]
similar_braveheart

[110,
 3842,
 2887,
 2695,
 3337,
 3579,
 3522,
 666,
 2783,
 3870,
 749,
 3126,
 2028,
 2785,
 3570]

In [64]:
movies[movies['movie_id'].isin(similar_braveheart)]

,movie_id,title,genre
108,110,Braveheart (1995),Action|Drama|War
660,666,All Things Fair (1996),Drama
739,749,"Man from Down Under, The (1943)",Drama
1959,2028,Saving Private Ryan (1998),Action|Drama|War
2626,2695,"Boys, The (1997)",Drama
2714,2783,"Tomb of Ligeia, The (1965)",Horror
2716,2785,Tales of Terror (1962),Horror
2818,2887,Simon Sez (1999),Drama
3057,3126,"End of the Affair, The (1955)",Drama
3268,3337,I'll Never Forget What's 'is Name (1967),Comedy|Drama


함수화해서 표현해봤습니다. 이번엔 그냥 비슷한 영화를 찾고싶다면 제목만 쳐도 됩니다. 

In [89]:
def get_similar_movie(movie_title : str):
    movie_id = movies[movies['title'] == (movie_title)].movie_id.values.tolist()
    movie_idd = movie_id_to_idx[movie_id[0]]
    similar_movie = als_model.similar_items(movie_idd)
    similar_movie = [idx_to_movie_id[i[0]] for i in similar_movie]
    return movies[movies['movie_id'].isin(similar_movie)]

In [90]:
get_similar_movie('American Beauty (1999)')

,movie_id,title,genre
739,749,"Man from Down Under, The (1943)",Drama
954,966,"Walk in the Sun, A (1945)",Drama
1728,1787,Paralyzing Fear: The Story of Polio in America...,Documentary
2522,2591,Jeanne and the Perfect Guy (Jeanne et le gar�o...,Comedy|Romance
2637,2706,American Pie (1999),Comedy
2789,2858,American Beauty (1999),Comedy|Drama
2928,2997,Being John Malkovich (1999),Comedy
3138,3207,"Snows of Kilimanjaro, The (1952)",Adventure
3710,3779,Project Moon Base (1953),Sci-Fi
3820,3890,Back Stage (2000),Documentary


가장 인기있었던 American Beauty (1999)에는 이런 비슷한 영화가 있다고 평가받았습니다. 

## 추천받기 

이제 진짜 추천을 한번 받아봅시다.

In [94]:
user = user_to_idx['hayaseleu']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended
recomended_movie_id = [idx_to_movie_id[i[0]] for i in movie_recommended]
recomended_movie_id 




[3114,
 527,
 589,
 1265,
 1233,
 145,
 3082,
 588,
 2467,
 1873,
 2702,
 3405,
 1584,
 1913,
 1049,
 493,
 3450,
 940,
 590,
 1408]

In [95]:
movies[movies['movie_id'].isin(recomended_movie_id )]

,movie_id,title,genre
143,145,Bad Boys (1995),Action
489,493,Menace II Society (1993),Action|Crime|Drama
523,527,Schindler's List (1993),Drama|War
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
586,590,Dances with Wolves (1990),Adventure|Drama|Western
928,940,"Adventures of Robin Hood, The (1938)",Action|Adventure
1035,1049,"Ghost and the Darkness, The (1996)",Action|Adventure
1214,1233,"Boat, The (Das Boot) (1981)",Action|Drama|War
1245,1265,Groundhog Day (1993),Comedy|Romance


알라딘을 추천해준 배경에 어떤 영화들이 기여했는지 알아봅시다.  알라딘의 movie_id 는 588 입니다. 

In [97]:
aladdin = movie_id_to_idx[588]
explain = als_model.explain(user, csr_data, itemid=aladdin)
# 이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다. 
# 어떤 영화들이 이 추천에 얼마나 기여하고 있는 걸까요?

[(idx_to_movie_id[i[0]], i[1]) for i in explain[1]]


[(1, 0.05422749232511039),
 (318, 0.0276626004637993),
 (110, 0.0064652348797863924),
 (2028, -0.002064740556360165),
 (480, -0.009926529888682964)]

토이스토리가 큰 영향을 미친것을확인할 수 있습니다. 아무래도 같은 애니메이션 작품이라서 그런것같습니다. 

## 소고

* 과적합                      
제가 좋아한다고 추가한 영화들의 내적을 1과 가깝게 끌어올리다보니 모델이 다소 과적합된것같습니다.
다른 영화들끼리는 그렇게 높은 값을 가지는걸 찾기가 힘들었던것같습니다. 
factors=500, regularization=0.01, use_gpu=False, iterations=200

* movie_id, movie_idx, movie_title                    
지금 movie 관련된 수치가 id,idx, title 세가지다 보니 다소 복잡합니다.                 
movie_id는 중간중간에 빠진값이 있기도 해서 idx를 위주로 모델을 적합시켰고 id를 통해 영화를 구별하고           
마지막으로 영화제목을 찾으니 다소 복잡해졌습니다. 그래도 get_similar_movie함수는 바로 title까지 뽑을수있도록      
만들어보았는데 정작 그렇게하자 얼마나 유사한지를 보여주는 수치가 빠져버려서 더 이상해졌습니다.

* merge                      
위의 복잡함 때문에 한번 merge를 통해서 해보려고도 했는데 자꾸 코드가 꼬이는 바람에           
한번만 사용하고 제대로 써보진않았습니다.
다음에는 더 활용해보도록 하겠습니다.
